In [ ]:
!curl  http://nightscout-data:3001/api/dumpcsv/ -o ../../data/nightscout/data.csv

In [ ]:
!ls ../../data/nightscout

In [ ]:
import os
data_dir='../../data/nightscout'
fname = os.path.join(data_dir,'data.csv')
f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

print(header, len(lines))
print(lines[0])

glucose_index = 0
keep_from_index = 2


In [ ]:
import numpy as np
len_of_each_line = len(lines[1].split(','))

data = np.zeros((len(lines),(len_of_each_line-keep_from_index)))

deletes = []
for i,l in enumerate(lines):
    if l!='':
        data[i,:]  = [float(x) for x in l.split(',')[keep_from_index:]]
    else:
        deletes.append(i)
        print("delete ",i)
print(deletes)
data = np.delete(data,deletes, axis=0)
print("data sanity check", data[-20:])
print(len(data))

In [ ]:
print("backup data in separate object")
datab = np.copy(data)
print(datab[108:115])

In [ ]:
def get_original_low_data(low_data_map, index, data_ = datab, num_before=24, num_after=5, lookahead=2):
    print("looking up data for low_data_map", low_data_map[index], "raw data index is ", low_data_map[index][1] )
    return data_[low_data_map[index][1]-num_before-lookahead : low_data_map[index][1]+num_after]

In [ ]:
up = True
low_count = 0
low_index = []
low_data_ = []
low_data = []
low_data_map = []
normal_data = []
low_threshold = 80
'''
The way this works is we'll use measures_before to predict. but lookahead is the gap between the last measure_before and the actual low event
so this is handy if you want to predict based on the last 2 hours you have a low in the next 10 minutes, 
you would set lookahead to 2 and measure before to 24
'''
measures_before = 24
lookahead = 2
low_data_index = 0
for index, i in enumerate(datab):
    if i[0] <= low_threshold and up:
        low_count+=1
        up = False
        low_index.append(index)
        if index > measures_before + lookahead:
            low_data_.append(datab[index-measures_before-lookahead:index-lookahead])
            low_data_map.append([low_data_index, index])
            low_data_index+=1

    if i[0] >low_threshold and not up:
        up = True

continuous = 0
for index, i in enumerate(datab):
    if i[0] <=low_threshold :
        continuous = 0
    if i[0] > low_threshold :
        continuous+=1
    if continuous > measures_before:
        normal_data.append(datab[index-measures_before-lookahead:index-lookahead])

#adding index to low_data so we can inspect it later
# print(len(low_data), low_data_[0])
# print("map:", low_data_map[0])
# print(datab[low_data_map[0][1]-5:low_data_map[0][1]])
# print(low_data_map)
np.random.shuffle(low_data_map)
np.random.shuffle(normal_data)
for i in range(len(low_data_map)):
    low_data.append(low_data_[low_data_map[i][0]])
# del low_data_

print("these should match if the mapping is correct.")
print(low_data_map[2])
print(low_data[2].shape, low_data[2])
print("ORIGINAL: \n", get_original_low_data(low_data_map, 2, data_=datab, num_before=measures_before+lookahead))


In [ ]:

#build the data set

splits = [.75,.2,.05]
norm_multiplier = 2
if(sum(splits) != 1):
    raise ValueError('splits must add up to 1')

train_low_i = round(len(low_data)*splits[0])
val_low_i = [train_low_i, train_low_i+round(len(low_data)*splits[1])]
test_low_i = [val_low_i[1], val_low_i[1]+round(len(low_data)*splits[2])]

train_norm_i = train_low_i * norm_multiplier
val_norm_i = [train_norm_i, val_low_i[1] * norm_multiplier]
test_norm_i = [val_norm_i[1], test_low_i[1] * norm_multiplier]

#training data
train_data_low = np.array(low_data[:train_low_i])
train_data_norm = np.array(normal_data[:train_norm_i])

train_labels_low = np.full((train_low_i,), 1)
train_labels_norm = np.full((train_norm_i,), 0)

train_data = np.concatenate((train_data_low, train_data_norm), axis=0)
train_labels = np.concatenate((train_labels_low, train_labels_norm), axis=0)
#validation data
val_data_low = np.array(low_data[val_low_i[0]:val_low_i[1]])
val_data_norm = np.array(normal_data[val_norm_i[0]:val_norm_i[1]])

val_labels_low = np.full((len(val_data_low),), 1)
val_labels_norm = np.full((len(val_data_norm),), 0)

val_data = np.concatenate((val_data_low, val_data_norm), axis=0)
val_labels = np.concatenate((val_labels_low, val_labels_norm), axis=0)
#test data
test_data_low = np.array(low_data[test_low_i[0]:test_low_i[1]])
test_data_norm = np.array(normal_data[test_norm_i[0]:test_norm_i[1]])
test_labels_low = np.full((len(test_data_low),), 1)
test_labels_norm = np.full((len(test_data_norm),), 0)
test_data = np.concatenate((test_data_low, test_data_norm), axis=0)
test_labels = np.concatenate((test_labels_low, test_labels_norm), axis=0)
print("*** LOWS ***")
print("training low data indexes: ","000","->", train_low_i)
print("validati low data indexes: ", val_low_i[0],"->",val_low_i[1])
print("testing  low data indexes: ", test_low_i[0],"->",test_low_i[1])

print("*** NORMS ***")
print("training norm data indexes: ","000","->", train_norm_i)
print("validati norm data indexes: ", val_norm_i[0],"->",val_norm_i[1])
print("testing  norm data indexes: ", test_norm_i[0],"->",test_norm_i[1])

print("*** COMBINED ***")
print("training: ", train_data.shape, train_labels.shape)
print("validati: ", val_data.shape, val_labels.shape)
print("test.   : ", test_data.shape, test_labels.shape)
# train_data = np.concatenate((train_data_low, train_data_normal), axis=0)
# train_labels = np.concatenate((train_labels_low, train_labels_normal), axis=0)

# print(train_data[10], train_labels[10], "\nOG:\n", get_original_low_data(low_data_map, 10))
# train_data_n = normal_data[]


# validation_data = np.array([(low_data[round(len(low_data)*.75):], np.full((len(low_data)-len(training_data[0]),), 1))])
# print(len(low_data[:round(len(low_data)*.75)]))
# print(len( np.full((round(len(low_data)*.75),), 1)))
# print(len())
# training_data  = np.concatenate((training_data, normal_data[:len(training_data)*2]), axis=0)
# validation_data  = np.concatenate((validation_data, normal_data[:len(validation_data)*2]), axis=0)

In [ ]:
#normalize/scale data
#we will create standard scaler 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_data_flat = train_data.reshape(-1,3)
# print(train_data_flat)
# print(train_data[0])
scaler.fit(train_data_flat)
print("scaler: std: ", scaler.var_, "\n std: ", data[:25000].std(axis=0))
print("\nscaler: mean: ", scaler.mean_, "\n mean: ", data[:25000].mean(axis=0))

for i in range(len(train_data)):
    train_data[i] = scaler.transform(train_data[i])
for i in range(len(val_data)):
    val_data[i] = scaler.transform(val_data[i])

In [ ]:
print("data summary")
print(train_data.shape, "\n", train_data)
print(val_data.shape, "\n", val_data)
print(test_data.shape, "\n", test_data)

In [ ]:
#build model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam

In [ ]:
#exploratory model with train/val split. find poitn of overfit
model = Sequential()
model.add(layers.GRU(32,  return_sequences=True, input_shape=(None, train_data.shape[-1])))
model.add(layers.GRU(64, activation='relu'))
model.add(layers.Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer='adam')
history = model.fit(train_data, train_labels,epochs=100, validation_data=(val_data, val_labels)) 

In [ ]:
#graph loss/val loss
import matplotlib.pyplot as plt
import pandas as pd

def plot_loss_accuracy(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, max(1, historydf.values.max())))
    loss = history.history['loss'][-1]
    acc = history.history['accuracy'][-1]
    plt.title('Loss: %.3f, Accuracy: %.3f' % (loss, acc))

def plot_loss(history):
    historydf = pd.DataFrame(history.history, index=history.epoch)
    plt.figure(figsize=(8, 6))
    historydf.plot(ylim=(0, historydf.values.max()))
    plt.title('Loss: %.3f' % history.history['loss'][-1])
    
%matplotlib inline

In [ ]:
plot_loss(history)

In [ ]:
#dropout network, exploratory with train/val/test
model_with_dropout = Sequential()
#need reset_after for GRU compatibility with tfjs as this model will run in node
model_with_dropout.add(layers.GRU(32, reset_after=False,  return_sequences=True, input_shape=(None, train_data.shape[-1])))
model_with_dropout.add(layers.GRU(64, reset_after=False, dropout=0.2, activation='relu'))
model_with_dropout.add(layers.Dense(1, activation="sigmoid"))
model_with_dropout.compile(loss="binary_crossentropy", optimizer=Adam(.0005))
history_with_dropout = model_with_dropout.fit(train_data, train_labels,batch_size=2, epochs=100, validation_data=(val_data, val_labels))


In [ ]:
#PRODUCTION network

#use hyperparams and epochs from exploratory with all data for production model

all_data = np.concatenate((train_data, val_data), axis=0)
all_labels = np.concatenate((train_labels, val_labels), axis=0)
model_with_dropout = Sequential()
model_with_dropout.add(layers.GRU(32, reset_after=False,  return_sequences=True, input_shape=(None, train_data.shape[-1])))
model_with_dropout.add(layers.GRU(64, reset_after=False, dropout=0.2, activation='relu'))
model_with_dropout.add(layers.Dense(1, activation="sigmoid"))
model_with_dropout.compile(loss="binary_crossentropy", optimizer=Adam(.0005))
history_with_dropout = model_with_dropout.fit(all_data, all_labels, batch_size=2, epochs=28, validation_data=(val_data, val_labels))

In [ ]:
plot_loss(history)
plot_loss(history_with_dropout)


In [ ]:

#make LSTM capable with batch of 1
#thanks: https://machinelearningmastery.com/use-different-batch-sizes-training-predicting-python-keras/
model_p = Sequential()
model_p.add(layers.GRU(32,  reset_after=False,  return_sequences=True, input_shape=(None, data.shape[-1]))) #interestingly you don't give it all the samples in shape
model_p.add(layers.GRU(64, reset_after=False,  activation='relu'))
model_p.add(layers.Dense(1, activation="sigmoid"))
#copy weights from trained model
old_weights = model_with_dropout.get_weights()

model_p.set_weights(old_weights)
model_p.compile(loss="binary_crossentropy", optimizer='adam')


In [ ]:
#some tests
wrong = 0
right = 0
for i in range(2000,10000):
    testone = np.expand_dims(scaler.transform(normal_data[i]), axis=0)
    # print(ss.shape, ss)
    result = model_p.predict(testone)
    if result > 0.5:
        wrong+=1
    else:
        right+=1
print(wrong, right)

In [ ]:

def predict_low(sequence):
    if(sequence.shape != (24,3)):
        raise exception("bad sequence", sequence.shape)
    testone = np.expand_dims(scaler.transform(sequence), axis=0)
    result = model_p.predict(testone)
    return result


In [ ]:
print(predict_low(get_original_low_data(low_data_map, -3)[:24]))
get_original_low_data(low_data_map, -3)


In [ ]:
#save the network
model_p.save(os.path.join(data_dir,'model_classify_no_reset_after.h5'))

In [ ]:
#save the scaler. need to scale new readings on prediction 
from joblib import dump, load
dump(scaler, os.path.join(data_dir,'scaler_model_classify.bin'), compress=True)

In [ ]:
print("mean: ", scaler.mean_)
print("var:  ", scaler.var_)